In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import cv2
import keras
from sklearn.metrics import confusion_matrix, classification_report 

In [ ]:
image_dir=Path('preprocessed')

In [ ]:
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df1 = pd.concat([filepaths, labels], axis=1)

In [ ]:

# # Import label encoder
# from sklearn import preprocessing
  
# # label_encoder object knows how to understand word labels.
# label_encoder = preprocessing.LabelEncoder()
  
# # Encode labels in column 'species'.
# image_df1['Label']= label_encoder.fit_transform(image_df1['Label'])
  
# image_df1['Label'].unique()

In [ ]:
image_df1

In [ ]:
image_df1.to_csv('image_labels.csv',index=False) 

In [ ]:
train_df, test_df = train_test_split(image_df1, train_size=0.75, shuffle=True, random_state=7)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(128, 128),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=20,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(128, 128),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=20,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(128, 128),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=20,
    shuffle=False
)

In [ ]:
import matplotlib.pyplot as plt

batch = next(train_images)
# for i in range(batch[0].shape[0]):
plt.imshow(batch[0][9], cmap='gray')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, regularizers
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

model = tf.keras.models.Sequential()
# Modify the input shape to have 1 channel
inputs = tf.keras.Input(shape=(128, 128, 1))

# Preprocessing
model.add(layers.Resizing(height=128, width=128, interpolation='bilinear'))
model.add(layers.Rescaling(scale=1./255))

# Data Augmentation
# model.add(layers.RandomFlip(mode='horizontal_and_vertical'))
# model.add(layers.RandomRotation(factor=.2))

# First Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(5, 5),
          padding='same', input_shape=(128, 128, 1)))
model.add(layers.Activation(tf.keras.activations.relu))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(.2))

# Second Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(layers.Activation(tf.keras.activations.relu))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(.2))

# Third Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(layers.Activation(tf.keras.activations.relu))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(.2))

# Feed forward
model.add(layers.Flatten())
model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(
    0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(layers.Activation(tf.keras.activations.relu))
model.add(layers.Dropout(.2))
model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(
    0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(layers.Activation(tf.keras.activations.relu))
model.add(layers.Dropout(.2))
model.add(layers.Dense(42))
model.add(layers.Activation(tf.keras.activations.softmax))


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
         tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
    ]
)

In [ ]:
model.save('model')

In [ ]:
from keras.models import load_model

# Load the saved model
model = load_model('model')

In [ ]:
# Use the model to make predictions on new data
predictions = model.predict(test_images)